In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os, cv2, random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm    #Helps in vis
from random import shuffle 
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
%matplotlib inline 

In [ ]:
import keras
from keras import applications
from keras import optimizers
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,GlobalMaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import Callback,ModelCheckpoint
from keras.applications import VGG16
from keras.applications import ResNet50
from keras.models import load_model
from keras.preprocessing.image import load_img

In [ ]:
val_size = 0.5
random_state = 1972
batch_size = 32
epochs= 10
num_classes = 2
sample_size = 20000
path = '../input/dogs-vs-cats-redux-kernels-edition/'
train_dir = path+'train/'
test_dir =  path+'test/'
sample_path='../input/test-data/test_data/'
sample_test_dir=sample_path+'test_data/'
img_size = 224
resnet_weight_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
train_images = os.listdir(train_dir)[0:sample_size]
test_images= os.listdir(test_dir)
sample_test_images=os.listdir(sample_test_dir)

In [ ]:
def create_pet_label_sample(img):
    #pet = img.split('.')[-3]
    pet = img[:3]
    if pet == 'cat': return [1,0]
    elif pet == 'dog': return [0,1]

def create_pet_label_train(img):
    pet = img.split('.')[-3]
    #pet = img[:2]
    if pet == 'cat': return [1,0]
    elif pet == 'dog': return [0,1]

In [ ]:
def process_images(image_list, image_dir,image_type):
    data_df = []
    for img in tqdm(image_list):
        path = os.path.join(image_dir,img)
        if image_type=='train':
            label = create_pet_label_train(img)
        if image_type=='sample':
            label = create_pet_label_sample(img)    
        else:
            label = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (img_size,img_size))
        data_df.append([np.array(img),np.array(label)])
    shuffle(data_df)
    return data_df

In [ ]:
sample_test=process_images(sample_test_images, sample_test_dir,'sample')

## Generating arrays of train and sample test image data

In [ ]:
Xt = np.array([i[0] for i in sample_test]).reshape(-1,img_size,img_size,3)
yt = np.array([i[1] for i in sample_test])

## Loading pretrained model from file

In [ ]:
model=load_model('../input/dogs-vs-cats-capstone-final-rn-50/resnet50.h5')

## Generate number of errors in predictions on sample test data

In [ ]:
#get the predictions for the test data
predicted_classes = model.predict_classes(Xt)
#get the indices to be plotted
y_true = np.argmax(yt,axis=1)
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

In [ ]:
target_names = ["Class {}:".format(i) for i in range(num_classes)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

## Print misclassified images

In [ ]:
# Get the filenames
fnames=sample_test_images
# Get the label to class mapping from the generator
label2index = {'cat': 0, 'dog': 1}
 # Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
ground_truth = np.argmax(yt,axis=1)
# Show the errors
predictions = model.predict(Xt)
predicted_class = np.argmax(predictions,axis=1)
errors = np.where(predicted_class != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),len(Xt)))
#f, ax = plt.subplots(5,5, figsize=(15,15))
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
    title = 'y_true:{},Original label:{}, Prediction :{}, confidence : {:.3f}'.format(ground_truth[i],
        fnames[errors[i]][:3],
        pred_label,
        predictions[errors[i]][pred_class])
    original = load_img('{}/{}'.format(sample_test_dir,fnames[errors[i]]))
    plt.figure(figsize=[5,5])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()

In [ ]:
f, ax = plt.subplots(5,5, figsize=(15,15))
for i,data in enumerate(sample_test[:25]):
    img_num = data[1]
    img_data = data[0]
    orig = img_data
    data = img_data.reshape(-1,img_size,img_size,3)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: 
        str_predicted='Dog'
    else: 
        str_predicted='Cat'
    ax[i//5, i%5].imshow(orig)
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_title("Predicted:{}".format(str_predicted))    
plt.show()